In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import torch
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader

import sys
sys.path.append('../')

from utils.metrics import RestMexMetrics
from utils.config import setConfig

device = setConfig()
metrics = RestMexMetrics()

Usando MPS: mps
Test correcto: mps


In [2]:
df = pd.read_csv(r'../data/train/train.csv')
audf = pd.read_csv(r'../data/augmented/train.csv')
data = pd.concat([df, audf], ignore_index=True)

data['Title'] = data['Title'].astype(str)
data['Review'] = data['Review'].astype(str)
data['Town'] = data['Town'].astype(str)
data['Region'] = data['Region'].astype(str)
data['Type'] = data['Type'].astype(str)
data['Polarity'] = data['Polarity'].astype(int)

In [3]:

train, test = train_test_split(data, test_size=0.20, random_state=42)

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

X_test = '<title>' + test['Title'] + '<title> <review>' + test['Review'] + '<review>'
y_test = test['Polarity']

Train shape: (170040, 6)
Test shape: (42511, 6)


In [ ]:
from utils.run import predict_sentiment

## Usando Tabularisai

In [5]:
model_name = "tabularisai/multilingual-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
model = model.to(device)
y_test_pred = predict_sentiment(X_test.to_numpy(), model, tokenizer, device.type, batch_size=16)

Predicting sentiment: 100%|██████████| 2657/2657 [07:39<00:00,  5.78it/s]


In [8]:
report = classification_report(y_test, y_test_pred, target_names=test['Polarity'].astype(int).unique(), output_dict=True)
report = pd.DataFrame(report)
f1 = report[y_test.unique()].loc['f1-score'].to_dict()

ResP_k = metrics.TypeScore(f1)

ResP_k

0.4341778822022997

## Ahora usando Finetuning

In [5]:
!ls ../models/tabularisai-distilbert/

config.json             tokenizer.json          vocab.txt
model.safetensors       tokenizer_config.json
special_tokens_map.json training_args.bin


In [4]:
path = '../models/tabularisai-distilbert/'
model = AutoModelForSequenceClassification.from_pretrained(path) 
tokenizer = AutoTokenizer.from_pretrained(path)

In [ ]:
model = model.to(device)
y_test_pred = predict_sentiment(X_test.to_numpy(), model, tokenizer, device.type, batch_size=16)

Predicting sentiment: 100%|██████████| 2657/2657 [07:56<00:00,  5.58it/s]


In [11]:
# Accuracy

from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(y_test, y_test_pred)
print(f"Accuracy: {accuracy:.4f}")
f1 = f1_score(y_test, y_test_pred, average='weighted')
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.7761
F1 Score: 0.7656


In [12]:
report = classification_report(y_test, y_test_pred, target_names=test['Polarity'].astype(int).unique(), output_dict=True)
report = pd.DataFrame(report)
f1 = report[y_test.unique()].loc['f1-score'].to_dict()

ResP_k = metrics.TypeScore(f1)

ResP_k

0.6346492447794829